In [1]:
import json

import pandas as pd
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

In [ ]:
with open('db_curated_server_info.txt', 'r') as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()  
    if line and not line.startswith('#'):  
        key, value = line.split('=', 1)  
        config[key] = value.strip()  

server_name = config.get('server_name')
srever_port = int(config.get('srever_port'))
ssh_password = config.get('ssh_password')
ssh_username = config.get('ssh_username')
db_adress = config.get('db_adress')
db_port = int(config.get('db_port'))

In [2]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

46825


In [3]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [4]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [21]:
query = "SELECT * FROM histone"
cursor.execute(query)
cursor.fetchall()
", h.".join([i[0] for i in cursor.description])

'id, h.level, h.taxonomic_span, h.taxonomic_span_id, h.description, h.parent'

In [18]:
query = "SELECT * FROM histone_description"
cursor.execute(query)
cursor.fetchall()
", d.".join([i[0] for i in cursor.description])

'id, d.summary, d.taxonomy, d.genes, d.evolution, d.expression, d.knock_out, d.function, d.sequence, d.localization, d.deposition, d.structure, d.interactions, d.disease, d.caveats'

In [20]:
query = "SELECT * FROM alternative_name"
cursor.execute(query)
cursor.fetchall()
", an.".join([i[0] for i in cursor.description])

'id, an.name, an.taxonomy, an.gene, an.splice, an.histone'

In [22]:
query = "SELECT * FROM publication"
cursor.execute(query)
cursor.fetchall()
", p.".join([i[0] for i in cursor.description])

'id, p.title, p.doi, p.author, p.year'

In [6]:
query = "SELECT * FROM histone WHERE id='H3.3'"
cursor.execute(query)
# for i in cursor:
#     print(i)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,H3.3,variant_group,Eukaryotes,2759,31,H3


In [10]:
query = "SELECT id FROM histone WHERE parent IS NULL"
cursor.execute(query)
types = cursor.fetchall()
types

[('Archaeal',), ('H1',), ('H2A',), ('H2B',), ('H3',), ('H4',), ('Viral',)]

In [16]:
query = "SELECT * FROM sequence s RIGHT JOIN sequence_has_publication sp ON s.accession = sp.sequence_accession"
cursor.execute(query)
sequence_df = pd.DataFrame(
    cursor.fetchall(), columns=[i[0] for i in cursor.description]
)
# sequence_df
sequence_df[
    ["publication_id", "accession", "variant", "variant_under_consideration", "organism", "sequence"]
].to_csv("publications.csv")
# .groupby(["publication_id"])["accession"].apply(
#     lambda x: [] if x.dropna().empty else list(x.unique())
# ).reset_index().to_csv('publications.csv')

In [140]:
query = "SELECT * FROM sequence_has_publication"
cursor.execute(query)
cursor.fetchall()
[i[0] for i in cursor.description]

['sequence_accession', 'publication_id']

In [6]:
cursor.close()
conn.close()
tunnel.stop()